In [3]:
import os
import gc
import torch

import transformers
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, BitsAndBytesConfig
from datasets import load_dataset
from peft import LoraConfig, PeftModel, get_peft_model, prepare_model_for_kbit_training
from trl import DPOTrainer
import bitsandbytes as bnb
# from google.colab import userdata
import wandb

In [4]:
device = "cuda"

In [5]:
model = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-v0.1")

config.json: 100%|██████████| 571/571 [00:00<00:00, 319kB/s]
model.safetensors.index.json: 100%|██████████| 25.1k/25.1k [00:00<00:00, 16.2MB/s]
model-00001-of-00002.safetensors: 100%|██████████| 9.94G/9.94G [12:53<00:00, 12.9MB/s]
model-00002-of-00002.safetensors: 100%|██████████| 4.54G/4.54G [05:47<00:00, 13.1MB/s]
generation_config.json: 100%|██████████| 116/116 [00:00<00:00, 2.18kB/s]


In [6]:
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-v0.1")

tokenizer_config.json: 100%|██████████| 967/967 [00:00<00:00, 1.71MB/s]
tokenizer.model: 100%|██████████| 493k/493k [00:00<00:00, 18.3MB/s]
tokenizer.json: 100%|██████████| 1.80M/1.80M [00:00<00:00, 17.5MB/s]
special_tokens_map.json: 100%|██████████| 72.0/72.0 [00:00<00:00, 136kB/s]


In [7]:
prompt = "My favourite condiment is"

In [8]:
model_inputs = tokenizer([prompt], return_tensors="pt").to(device)

In [9]:
model.to(device)

MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): MistralRotaryEmbedding()
        )
        (mlp): MistralMLP(
          (gate_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): MistralRMSNorm()
        (post_attention_layernorm): MistralRMSNorm()
      )
    )
    (norm): MistralRMSNorm()
  

In [10]:
generated_ids = model.generate(**model_inputs, max_new_tokens=100, do_sample=True)
tokenizer.batch_decode(generated_ids)[0]

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


'<s> My favourite condiment is most definitely mayo – and the spicy the better.\n\nThis is the basic recipe that I make for my everyday mayo (without chilli), but I make it double strength (with an extra egg) to add to stir fries!\n\nIt really is so quick to make – you’ll never buy bottled again. You’ll have more flexibility with adding garlic and chilli according to your taste (rather than the company’s).\n\nIt keeps in the fr'